# Testing the Globus API

## Define variables

In [1]:
CLIENT_ID = 'e8954f4a-23c6-4ffc-8eaa-747fda1f9351'
blueTeamEndpointID = 'eed5aba8-6bc9-11e7-a9c6-22000bf2d287'
redTeamEndpointID = '3435b504-6bc4-11e7-a9c6-22000bf2d287'
rootTransferFolderName = "Red_Team_Stuff"
transferFile = 'Transparent Aluminum.zip'

## Import Globus,  connect to app and start TransferClient

In [2]:
import globus_sdk

client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
client.oauth2_start_flow(refresh_tokens=True)

print('Please go to this URL and login: {0}'
      .format(client.oauth2_get_authorize_url()))

get_input = getattr(__builtins__, 'raw_input', input)
auth_code = get_input('Please enter the code here: ').strip()
token_response = client.oauth2_exchange_code_for_tokens(auth_code)

# let's get stuff for the Globus Transfer service
globus_transfer_data = token_response.by_resource_server['transfer.api.globus.org']
# the refresh token and access token, often abbr. as RT and AT
transfer_rt = globus_transfer_data['refresh_token']
transfer_at = globus_transfer_data['access_token']
expires_at_s = globus_transfer_data['expires_at_seconds']

authorizer = globus_sdk.RefreshTokenAuthorizer(
    transfer_rt, client, access_token=transfer_at, expires_at=expires_at_s)

# and try using `tc` to make TransferClient calls.
tc = globus_sdk.TransferClient(authorizer=authorizer)

Please go to this URL and login: https://auth.globus.org/v2/oauth2/authorize?code_challenge=XjCbDGIMuwIlZKjBHwzOTfOE97fU4iaXveCLmAYJtSw&state=_default&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&response_type=code&client_id=e8954f4a-23c6-4ffc-8eaa-747fda1f9351&scope=openid+profile+email+urn%3Aglobus%3Aauth%3Ascope%3Atransfer.api.globus.org%3Aall&code_challenge_method=S256&access_type=offline
Please enter the code here: H0XFZrPk81mX4BoGCt4HoacUVnJGpG


## Get a response from the endpoint

In [3]:
endpoint = tc.get_endpoint(blueTeamEndpointID)
print "Endpoint name: {}".format(endpoint["canonical_name"])

Endpoint name: u_cdtg35asyreyhnmqmhbzl5jab4#mdi_summit_blue_team


## List dirs and files in the Blue Team endpoint

In [4]:
blueTeamDirs = []
for response in tc.operation_ls(blueTeamEndpointID):
    print 'Name: {}, type: {}'.format(response['name'], response['type'])
    if response['type'] == 'dir':
        blueTeamDirs.append(response)

Name: .cache, type: dir
Name: Junk Folder, type: dir
Name: Red_Team_Stuff, type: dir
Name: .bash_logout, type: file
Name: .bashrc, type: file
Name: .profile, type: file
Name: 7707_photo.png, type: file
Name: 7887_photo.png, type: file
Name: 7997_photo.png, type: file
Name: globus-connect-server-repo_latest_all.deb, type: file
Name: helpme.txt, type: file
Name: small_text.txt, type: file


## Check if root transfer path exist otherwise create it

In [5]:
newRootPath = "./{}".format(rootTransferFolderName)
rootPathExists = False
for folder in blueTeamDirs:
    if folder['name'] == rootTransferFolderName:
        rootPathExists = True
if not rootPathExists: #Create an new dir in the blue team endpoint
    mkDirResponse = tc.operation_mkdir(blueTeamEndpointID, path=newRootPath)
    print mkDirResponse
else:
    print 'Root transfer path already exists'

Root transfer path already exists


## Create new dir in the root transfer dir with dt stamp

In [6]:
from time import gmtime, strftime
dtStamp = strftime("%Y-%m-%d %H:%M:%S", gmtime())
newTransferFolderName = dtStamp.replace(' ','_')
newFolderPath = '{}/{}'.format(newRootPath, newTransferFolderName)
mkDirResponse = tc.operation_mkdir(blueTeamEndpointID, path=newFolderPath)
print mkDirResponse

{
  "message": "The directory was created successfully", 
  "code": "DirectoryCreated", 
  "resource": "/operation/endpoint/eed5aba8-6bc9-11e7-a9c6-22000bf2d287/mkdir", 
  "DATA_TYPE": "mkdir_result", 
  "request_id": "kTJP00yrm"
}


## Transfer file to new folder

In [7]:
transferData = globus_sdk.TransferData(tc, redTeamEndpointID,
                                blueTeamEndpointID)
transferedFilePath = '{}/{}'.format(newFolderPath, transferFile)
transferData.add_item(transferFile, transferedFilePath)
transfer_result = tc.submit_transfer(transferData)
print transfer_result

{
  "submission_id": "c0a7e2a7-6c95-11e7-a9cb-22000bf2d287", 
  "code": "Accepted", 
  "resource": "/transfer", 
  "task_id": "c0a7e2a6-6c95-11e7-a9cb-22000bf2d287", 
  "DATA_TYPE": "transfer_result", 
  "request_id": "mTfUyuXlR", 
  "message": "The transfer has been accepted and a task has been created and queued for execution", 
  "task_link": {
    "href": "task/c0a7e2a6-6c95-11e7-a9cb-22000bf2d287?format=json", 
    "resource": "task", 
    "rel": "related", 
    "DATA_TYPE": "link", 
    "title": "related task"
  }
}


## Example - delete dir in endpoint

In [ ]:
deleteData = globus_sdk.DeleteData(tc, blueTeamEndpointID, recursive=True)
deleteData.add_item("Red_Team_Stuff")
delete_result = tc.submit_delete(deleteData)
print("task_id =", delete_result["task_id"])

## Example - text search on Globus endpoints

In [ ]:
for ep in tc.endpoint_search('Jan'):
    print(ep['display_name'])

## Example - Rename the new dir

In [ ]:
renamePathResponse = tc.operation_rename(blueTeamEndpointID, oldpath="./Jan's Folder", newpath="./Red_Team_Stuff")
print renamePathResponse